# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,50.56,55,62,12.01,CL,1698615687
1,1,san patricio,28.0170,-97.5169,89.55,58,64,11.48,US,1698615687
2,2,heilbad heiligenstadt,51.3833,10.1333,52.84,84,82,3.00,DE,1698615688
3,3,kas,36.2018,29.6377,71.38,67,100,3.47,TR,1698615688
4,4,taoudenni,22.6783,-3.9836,89.96,11,0,7.78,ML,1698615688


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    height=450,
    width=800,
    scale = 0.75,
    size = "Humidity",
    color = "City",
    alpha = 0.5
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [66]:
# Narrow down cities that fit criteria and drop any results with null values
refined_city_df = city_data_df.loc[(city_data_df["Wind Speed"] <4.5) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Max Temp"] >= 50) & (city_data_df["Max Temp"] <= 80)].dropna()

# Display sample data
refined_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
65,65,saint-francois,46.4154,3.9054,53.65,100,0,1.14,FR,1698615699
96,96,beroroha,-21.6667,45.1667,75.00,47,0,3.42,MG,1698615703
174,174,rreshen,41.7675,19.8756,59.97,69,0,4.45,AL,1698615714
206,206,kazanlak,42.6167,25.4000,53.17,69,0,1.25,BG,1698615719
266,266,tsiombe,-25.3000,45.4833,71.82,84,0,4.27,MG,1698615729
301,301,shimoda,34.6667,138.9500,55.11,65,0,2.82,JP,1698615735
412,412,clarence town,-32.5833,151.7833,70.14,37,0,3.65,AU,1698615755
499,499,jingzhou,30.3503,112.1903,63.72,76,0,4.03,CN,1698615767
508,508,laguna,38.4210,-121.4238,75.29,20,0,3.44,US,1698615769
521,521,xishan,27.6667,113.5000,62.51,85,0,0.98,CN,1698615771


### Step 3: Create a new DataFrame called `hotel_df`.

In [67]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = refined_city_df.loc[:,["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
65,saint-francois,FR,46.4154,3.9054,100,
96,beroroha,MG,-21.6667,45.1667,47,
174,rreshen,AL,41.7675,19.8756,69,
206,kazanlak,BG,42.6167,25.4000,69,
266,tsiombe,MG,-25.3000,45.4833,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [68]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat},{radius}"
    
    # Set query URL
    query = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:{lng},{lat},{radius}&bias=proximity:{lng},{lat}&limit=1&apiKey={geoapify_key}"
    # Make an API request using the query url
    response = requests.get(query)
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)

Starting hotel search
saint-francois - nearest hotel: Chez Lily
beroroha - nearest hotel: No hotel found
rreshen - nearest hotel: Vila Livio
kazanlak - nearest hotel: Хотел Казанлък
tsiombe - nearest hotel: No hotel found
shimoda - nearest hotel: ホテルウラガ
clarence town - nearest hotel: Erringhi Hotel
jingzhou - nearest hotel: G CHU
laguna - nearest hotel: Holiday Inn Express & Suites
xishan - nearest hotel: Dongfeng
sheohar - nearest hotel: No hotel found
               City Country      Lat       Lng  Humidity  \
65   saint-francois      FR  46.4154    3.9054       100   
96         beroroha      MG -21.6667   45.1667        47   
174         rreshen      AL  41.7675   19.8756        69   
206        kazanlak      BG  42.6167   25.4000        69   
266         tsiombe      MG -25.3000   45.4833        84   
301         shimoda      JP  34.6667  138.9500        65   
412   clarence town      AU -32.5833  151.7833        37   
499        jingzhou      CN  30.3503  112.1903        76   
50

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [69]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    color = "r",
    hover_cols=["City", "Country", "Hotel Name"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)